In [ ]:
import cvxpy as cvx
import numpy as np

def optimize_twoasset_portfolio(varA, varB, rAB):
    """Create a function that takes in the variance of Stock A, the variance of
    Stock B, and the correlation between Stocks A and B as arguments and returns 
    the vector of optimal weights
    
    Parameters
    ----------
    varA : float
        The variance of Stock A.
        
    varB : float
        The variance of Stock B.    
        
    rAB : float
        The correlation between Stocks A and B.
        
    Returns
    -------
    x : np.ndarray
        A 2-element numpy ndarray containing the weights on Stocks A and B,
        [x_A, x_B], that minimize the portfolio variance.
    
    """
    # TODO: Use cvxpy to determine the weights on the assets in a 2-asset
    # portfolio that minimize portfolio variance.
    
    cov = np.sqrt(varA)*np.sqrt(varB)*rAB
    
    x = cvx.Variable(2)
    
    P = np.array([[varA, cov], [cov, varB]])
    
    portfolio_variance = cvx.quad_form(x, P)
    
    objective = cvx.Minimize(portfolio_variance)
    
    constraints = [sum(x) == 1]
    
    problem = cvx.Problem(objective, constraints)
    
    min_value = problem.solve()
    xA,xB = x.value
    
    # return xA and xB
    return xA, xB

In [2]:

def optimize_portfolio(returns, index_weights, scale=.00001):
    """
    Create a function that takes the return series of a set of stocks, the index weights,
    and scaling factor. The function will minimize a combination of the portfolio variance
    and the distance of its weights from the index weights.  
    The optimization will be constrained to be long only, and the weights should sum to one.
    
    Parameters
    ----------
    returns : numpy.ndarray
        2D array containing stock return series in each row.
        
    index_weights : numpy.ndarray
        1D numpy array containing weights of the index.
        
    scale : float
        The scaling factor applied to the distance between portfolio and index weights
        
    Returns
    -------
    x : np.ndarray
        A numpy ndarray containing the weights of the stocks in the optimized portfolio
    """
    # TODO: Use cvxpy to determine the weights on the assets
    # that minimizes the combination of portfolio variance and distance from index weights
    
    # number of stocks m is number of rows of returns, and also number of index weights
    m = returns.shape[0]
    
    #covariance matrix of returns
    cov = np.cov(returns)
    
    # x variables (to be found with optimization)
    x = cvx.Variable(m)
    
    #portfolio variance, in quadratic form
    portfolio_variance = cvx.quad_form(x, cov)
    
    # euclidean distance (L2 norm) between portfolio and index weights
    distance_to_index = cvx.norm((x-index_weights), axis=0)
    
    #objective function
    objective = cvx.Minimize(portfolio_variance + scale * distance_to_index)
    
    #constraints
    constraints = [x >= 0, sum(x) == 1]

    #use cvxpy to solve the objective
    cvx.Problem(objective, constraints).solve()
    
    
    #retrieve the weights of the optimized portfolio
    x_values = x.value
    
    return x_values

In [3]:
def generate_dollar_volume_weights(close, volume):
    """
    Generate dollar volume weights.

    Parameters
    ----------
    close : DataFrame
        Close price for each ticker and date
    volume : str
        Volume for each ticker and date

    Returns
    -------
    dollar_volume_weights : DataFrame
        The dollar volume weights for each ticker and date
    """
    assert close.index.equals(volume.index)
    assert close.columns.equals(volume.columns)
    
    weighted = close*volume.values

    return weighted.div(weighted.sum(axis=1), axis=0)

In [4]:
def calculate_dividend_weights(dividends):
    """
    Calculate dividend weights.

    Parameters
    ----------
    dividends : DataFrame
        Dividend for each stock and date

    Returns
    -------
    dividend_weights : DataFrame
        Weights for each stock and date
    """
    #TODO: Implement function
    
    
    #Help: I missed something here.
    
    #this is the temporally accumulated dividends by ticker/col
    col_tot = dividends.cumsum(axis=0)
    
    #This is the accumulated sum of all dividends by date. 
    daily_cum_sum = dividends.cumsum(axis=1).sum(axis=1)
    
    #divide each days accumulated ticker dividend total by all previous accumulated dividend totals
    something = col_tot.div(daily_cum_sum, axis=0)
    
    
    return something.div(something.sum(axis=1), axis=0)


In [5]:
def generate_returns(prices):
    """
    Generate returns for ticker and date.

    Parameters
    ----------
    prices : DataFrame
        Price for each ticker and date

    Returns
    -------
    returns : Dataframe
        The returns for each ticker and date
    """
    #TODO: Implement function

    return (prices - prices.shift(1))/prices.shift(1)

In [6]:
def generate_weighted_returns(returns, weights):
    """
    Generate weighted returns.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date
    weights : DataFrame
        Weights for each ticker and date

    Returns
    -------
    weighted_returns : DataFrame
        Weighted returns for each ticker and date
    """
    assert returns.index.equals(weights.index)
    assert returns.columns.equals(weights.columns)
    
    

    return returns*weights.values

In [1]:
def calculate_cumulative_returns(returns):
    """
    Calculate cumulative returns.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date

    Returns
    -------
    cumulative_returns : Pandas Series
        Cumulative returns for each date
    """
    #TODO: Implement
    
    return (returns.sum(axis=1) +1).cumprod()

In [2]:
def tracking_error(benchmark_returns_by_date, etf_returns_by_date):
    """
    Calculate the tracking error.

    Parameters
    ----------
    benchmark_returns_by_date : Pandas Series
        The benchmark returns for each date
    etf_returns_by_date : Pandas Series
        The ETF returns for each date

    Returns
    -------
    tracking_error : float
        The tracking error
    """
    assert benchmark_returns_by_date.index.equals(etf_returns_by_date.index)
    
    #TODO: Implement function

    return np.sqrt(252)*(np.std((etf_returns_by_date - benchmark_returns_by_date), ddof=1))

In [3]:
def get_covariance_returns(returns):
    """
    Calculate covariance matrices.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date

    Returns
    -------
    returns_covariance  : 2 dimensional Ndarray
        The covariance of the returns
    """
    #TODO: Implement function
        
    return np.cov(returns.fillna(0.0).T)

In [4]:
def rebalance_portfolio(returns, index_weights, shift_size, chunk_size):
    """
    Get weights for each rebalancing of the portfolio.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date
    index_weights : DataFrame
        Index weight for each ticker and date
    shift_size : int
        The number of days between each rebalance
    chunk_size : int
        The number of days to look in the past for rebalancing

    Returns
    -------
    all_rebalance_weights  : list of Ndarrays
        The ETF weights for each point they are rebalanced
    """
    assert returns.index.equals(index_weights.index)
    assert returns.columns.equals(index_weights.columns)
    assert shift_size > 0
    assert chunk_size >= 0
    
    #TODO: Implement
    

    end = returns.shape[0]

    start = chunk_size

    step = shift_size

    weights_list = []

    #with a range loop we can start at the first rebalance lookback point and
    #calculate back to start
    #We can perform this lookback at each incremental step with step defined by shift_size for our 
    #number of days between each rebalance. 
    #So at each increment i, you are always looking back the chunk_size
    #And each increment i is equal to our shift size
    for i in range(start, end, step):

        
            current_period_returns = returns[(i-chunk_size):i]

            #get the covariance of the current return period
            cov_returns = get_covariance_returns(current_period_returns)

            #rebalance the weights according to current_period index_weights
            weights = get_optimal_weights(cov_returns, index_weights[(i-chunk_size):i].iloc[-1])

            weights_list.append(weights)

    return weights_list

In [5]:
def get_portfolio_turnover(all_rebalance_weights, shift_size, rebalance_count, n_trading_days_in_year=252):
    """
    Calculage portfolio turnover.

    Parameters
    ----------
    all_rebalance_weights : list of Ndarrays
        The ETF weights for each point they are rebalanced
    shift_size : int
        The number of days between each rebalance
    rebalance_count : int
        Number of times the portfolio was rebalanced
    n_trading_days_in_year: int
        Number of trading days in a year

    Returns
    -------
    portfolio_turnover  : float
        The portfolio turnover
    """
    assert shift_size > 0
    assert rebalance_count > 0
    
    total=0

    
    for j in range(0, rebalance_count):
        #sum across the absolute value of the trading weight differences between periods
        total += np.absolute(all_rebalance_weights[j+1] - all_rebalance_weights[j]).sum()

    #compute the num trading events per year 
    trading_events_year =( np.floor(n_trading_days_in_year/shift_size) )
    
    #compute turnover according to equation above
    portfolio_turnover = total/rebalance_count*trading_events_year

#     print(portfolio_turnover)
    return portfolio_turnover